# Лабораторная работа 5.2
**Тема.** Системы дифференциальных уравнений
<br>
**Цель.** Научиться строить фазовый портрет нормальной системы 2-го порядка в точке покоя, аналитически находить общее и частное решение линейных систем методами Лагранжа, Эйлера, Даламбера и контролировать результаты с помощью средств СКА.

## **Задание 1.** Для данных систем установите тип фазовых картин и изобразите их. Сравните найденные собственные значения и векторы матрицы системы с результатами, полученными в СКА.
Найдите общее решение системы и выделите фундаментальную систему решений. Сравните с результатами, полученными в СКА
Постройте в прямоугольной системе $ Oxy_{1}y_{2} $ y пространственную кривую, удовлетворяющую системе, и содержащую точку $(0,0,0)$.

In [6]:
x = var('x')
y_1 = function('y_1')(x)
y_2 = function('y_2')(x)

eq_1 = diff(y_1, x) == 3 * y_1 + 5 * y_2
eq_2 = diff(y_2, x) == y_1 + 7 * y_2

Для начала найдем собственные значения матрицы системы.

In [7]:
A = matrix([[3,5],[1,7]])
eigenvals = A.eigenvalues()
eigenvects = A.eigenvectors_right()
show(eigenvals) 
show(eigenvects)

[8, 2]

[(8, [(1, 1)], 1), (2, [(1, -1/5)], 1)]

Тип фазовой картины определется:
- Если собственные значения действительные и положительные → узел-источник (наш случай).
- Если собственные значения действительные и разного знака → седло.
- Если комплексные → фокус/центр (знак действительной части определяет направление).

In [8]:
sol = desolve_system([eq_1, eq_2], [y_1, y_2])
show(sol)

[y_1(x) == 1/6*(y_1(0) + 5*y_2(0))*e^(8*x) + 5/6*(y_1(0) - y_2(0))*e^(2*x),
 y_2(x) == 1/6*(y_1(0) + 5*y_2(0))*e^(8*x) - 1/6*(y_1(0) - y_2(0))*e^(2*x)]

Выделите фундаментальную систему решений.

In [9]:
lambda1 = eigenvals[0]
lambda2 = eigenvals[1]

v1 = vector(eigenvects[0][1][0])
v2 = vector(eigenvects[1][1][0])

Построим кривую в 3D

In [10]:
y1_expr = v1[0]*exp(lambda1*x) + v2[0]*exp(lambda2*x)
y2_expr = v1[1]*exp(lambda1*x) + v2[1]*exp(lambda2*x)

curve = parametric_plot3d((x, y1_expr, y2_expr), (x, -2, 2),
                           color='blue', thickness=2)
curve.show()

Graphics3d Object

## **Задание 2.** Решите систему методом исключений и сравните результат с ответом, полученным в СКА

In [1]:
x = var('x')
y_1 = function('y_1')(x)
y_2 = function('y_2')(x)

eq_1 = diff(y_1, x) == 7 * y_1 + y_2
eq_2 = diff(y_2, x) == 5 * y_1 + 3 * y_2

Исключим $ y_{2} $

In [2]:
y2_expr = solve(eq_1, y_2)[0].rhs()
show(y2_expr)

-7*y_1(x) + diff(y_1(x), x)

Подставим $y2$ во второе уравнение

In [4]:
eq_y1 = diff(y2_expr, x) == 5*y_1 + 3*y2_expr
eq_y1 = simplify(eq_y1)
show(eq_y1)

-7*diff(y_1(x), x) + diff(y_1(x), x, x) == -16*y_1(x) + 3*diff(y_1(x), x)

Решим уравнение с помощью `desolve`

In [6]:
sol_y1 = desolve(eq_y1, y_1)
show(sol_y1)

_K1*e^(8*x) + _K2*e^(2*x)

Теперь найдем $y_{2}$ через формулу $ y_{2} = y_{1}' - 7y_{1} $

In [7]:
y2_sol = diff(sol_y1, x) - 7*sol_y1
show(y2_sol)

_K1*e^(8*x) - 5*_K2*e^(2*x)

Проверим решение через `dsolve_system`

In [9]:
sol = desolve_system([eq_1, eq_2], [y_1, y_2])
show(sol)

[y_1(x) == 1/6*(5*y_1(0) + y_2(0))*e^(8*x) + 1/6*(y_1(0) - y_2(0))*e^(2*x),
 y_2(x) == 1/6*(5*y_1(0) + y_2(0))*e^(8*x) - 5/6*(y_1(0) - y_2(0))*e^(2*x)]

## **Задание 3.** Решите задачу Коши с помощью методов Лагранжа и Даламбера. Сравните с результатом, полученным в СКА. Сделайте чертеж.
$$
\begin{cases}
      \dot{x} = 2x - 2y \\
      \dot{y} = -4x
\end{cases}
$$
$$
x(0)=3, \space y(0)=1
$$

In [4]:
t = var('t')
x = function('x')(t)
y = function('y')(t)

**1. Метод Лагранжа**
</br>Составляем характеристическое уравнение для x:
$$
\ddot{x} - 2\dot{x} + 8x = 0
$$

In [7]:
char_eq = diff(x, t, 2) - 2*diff(x, t) - 8*x == 0
sol_lagrange_x = desolve(char_eq, x, ics=[0, 3, 4])
show(sol_lagrange_x)

5/3*e^(4*t) + 4/3*e^(-2*t)

Вычисляем $y$ через $x$

In [8]:
sol_lagrange_y = sol_lagrange_x - 1/2 * diff(sol_lagrange_x, t)
show(sol_lagrange_y)

-5/3*e^(4*t) + 8/3*e^(-2*t)

**2. Метод Даламбера** 
</br>Решаем через интегральные кривые / прямое интегрирование.
Используем соотношение 
$$
\frac{dx}{2x-2y} = \frac{dy}{-4x}
$$

In [9]:
dx = var('dx')
dy = var('dy')
system = [diff(x, t) == 2*x - 2*y, diff(y, t) == -4*x]

Получаем решение

In [10]:
sol_dalember = desolve_system(system, [x, y], ivar=t, ics=[0, 3, 1])
show(sol_dalember)

[x(t) == 5/3*e^(4*t) + 4/3*e^(-2*t), y(t) == -5/3*e^(4*t) + 8/3*e^(-2*t)]

Проверим решение через стандартный `desolve_system` (преобразование Лапласа)

In [11]:
eq_1 = diff(x, t) == 2*x - 2*y
eq_2 = diff(y, t) == -4*x

sol_laplace = desolve_system([eq_1, eq_2], [x, y])
show(sol_laplace)

[x(t) == 1/3*(2*x(0) - 1)*e^(4*t) + 1/3*(x(0) + 1)*e^(-2*t),
 y(t) == -1/3*(2*x(0) - 1)*e^(4*t) + 2/3*(x(0) + 1)*e^(-2*t)]